<h2 style="text-align: center; font-family: Verdana; font-size: 24px; font-style: normal; font-weight: bold; text-decoration: underline; text-transform: none; letter-spacing: 2px; color: darkred; background-color: #ffffff;"><br><br>NOVOZYMES ENZYME STABILITY PREDICTION<br><br> RESIDUE DEPTH W/ BIOPYTHON<br></h2>

<h5 style="text-align: center; font-family: Verdana; font-size: 12px; font-style: normal; font-weight: bold; text-decoration: None; text-transform: none; letter-spacing: 1px; color: black; background-color: #ffffff;">CREATED BY: DARIEN SCHETTLER</h5>

<br>

---

<br>

<center><div class="alert alert-block alert-danger" style="margin: 2em; line-height: 1.7em; font-family: Verdana;">
    <b style="font-size: 18px;">🛑 &nbsp; WARNING:</b><br><br><b>THIS IS A WORK IN PROGRESS</b><br>
</div></center>


<center><div class="alert alert-block alert-warning" style="margin: 2em; line-height: 1.7em; font-family: Verdana;">
    <b style="font-size: 18px;">👏 &nbsp; IF YOU FORK THIS OR FIND THIS HELPFUL &nbsp; 👏</b><br><br><b style="font-size: 22px; color: darkorange">PLEASE UPVOTE!</b><br><br>This was a lot of work for me and while it may seem silly, it makes me feel appreciated when others like my work. 😅
</div></center>

<br>

---

<br>

### OVERVIEW

<br>

<mark><b>Residue depth is the average distance of the atoms of a residue from the solvent accessible surface.</b></mark><br><br>

<b>1. Calculate PER-RESIDUE SASA</b><br>
<b>2. Calculate PER-RESIDUE Residue Depth</b><br>
<b>3. Calculate DeMaSk Directional Substitution Score (Impact of Single Point Mutation)</b><br>
<b>4. Then use DeMaSk as an 'importance' multiplier for the SASA and Residue Depth scores to break ties. This is required because both SASA and DeMaSk only operate on the wildtype sequence and don't consider the impact of the mutation</b><br>
<b>5. Finally we also add the change in SASA and Residue Depth as a 4/5th feature that can act similarily to the DeMaSk substitution by telling us which substitutions will impact the protein more (larger delta)</b>

<br>


<br>

<b>Imports and MSMS Install <i>(was NOT easy to find this)</i></b>

In [1]:
print("\n... IMPORTS STARTING ...\n")

!conda install -y -c bioconda msms

# Pip installs for competition specific stuff
from Bio.PDB.ResidueDepth import ResidueDepth
from Bio.PDB.PDBParser import PDBParser
from Bio.PDB.SASA import ShrakeRupley
!pip install -q biopandas
!pip install -q biopython
from scipy import stats
import pandas as pd
import numpy as np
import Bio
import sys
import os

print("\n\n... IMPORTS COMPLETE ...\n")


... IMPORTS STARTING ...

Solving environment: done

## Package Plan ##

  environment location: /opt/conda

  added / updated specs:
    - msms


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2022.9.24  |       ha878542_0         150 KB  conda-forge
    certifi-2022.9.24          |     pyhd8ed1ab_0         155 KB  conda-forge
    conda-22.9.0               |   py37h89c1867_1         960 KB  conda-forge
    msms-2.6.1                 |       h9ee0642_3          97 KB  bioconda
    ------------------------------------------------------------
                                           Total:         1.3 MB

The following NEW packages will be INSTALLED:

  msms               bioconda/linux-64::msms-2.6.1-h9ee0642_3

The following packages will be UPDATED:

  ca-certificates                    2022.6.15.2-ha878542_0 --> 2022.9.24-ha878542_0
  certifi                       

<br>

<b>Load Required Files</b>

In [2]:
test_wt_pdb_path = "../input/novozymes-enzyme-stability-prediction/wildtype_structure_prediction_af2.pdb"
test_df = pd.read_csv("../input/novozymes-enzyme-stability-prediction/test.csv")
test_df_w_extra_info = pd.read_csv("../input/new-novoesp-train-and-test-data/new_test.csv")
ss_df = pd.read_csv("../input/novozymes-enzyme-stability-prediction/sample_submission.csv")
test_df = test_df.merge(test_df_w_extra_info[["seq_id", "edit_type", "edit_idx", "wildtype_aa", "mutant_aa"]], on="seq_id", how="left").fillna(-1)
test_df["edit_idx"] = test_df["edit_idx"].astype(int)

# Wildtype test sequence 
wildtype_aa = "VPVNPEPDATSVENVALKTGSGDSQSDPIKADLEVKGQSALPFDVDCWAILCKGAPNVLQRVNEKTKNSNRDRSGANKGPFKDPQKWGIKALPPKNPSWSAQDFKSPEEYAFASSLQGGTNAILAPVNLASQNSQGGVLNGFYSANKVAQFDPSKPQQTKGTWFQITKFTGAAGPYCKALGSNDKSVCDKNKNIAGDWGFDPAKWAYQYDEKNNKFNYVGK"

test_df

,seq_id,protein_sequence,pH,data_source,edit_type,edit_idx,wildtype_aa,mutant_aa
0,31390,VPVNPEPDATSVENVAEKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,substitution,16,L,E
1,31391,VPVNPEPDATSVENVAKKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,substitution,16,L,K
2,31392,VPVNPEPDATSVENVAKTGSGDSQSDPIKADLEVKGQSALPFDVDC...,8,Novozymes,deletion,16,L,-1
3,31393,VPVNPEPDATSVENVALCTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,substitution,17,K,C
4,31394,VPVNPEPDATSVENVALFTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,substitution,17,K,F
...,...,...,...,...,...,...,...,...
2408,33798,VPVNPEPDATSVENVILKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,substitution,15,A,I
2409,33799,VPVNPEPDATSVENVLLKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,substitution,15,A,L
2410,33800,VPVNPEPDATSVENVNLKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,substitution,15,A,N
2411,33801,VPVNPEPDATSVENVPLKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,substitution,15,A,P


<br>

<b>Get Residue SASA</b>

In [3]:
pdb_parser = PDBParser(QUIET=1)

def get_sr_sasa(pdb_path, return_struct=False, pdb_identifier="AF70", sr_n_points=250):
    sr = ShrakeRupley(n_points=sr_n_points)
    struct = pdb_parser.get_structure(pdb_identifier, pdb_path)
    sr.compute(struct, level="R")
    if return_struct:
        return struct
    else:
        return [x.sasa for x in struct.get_residues()]
    
residue_idx_to_sasa = {i:_sasa for i, _sasa in enumerate(get_sr_sasa(test_wt_pdb_path))}
test_df["sasa"] = test_df["edit_idx"].apply(lambda x: residue_idx_to_sasa.get(x, 0.0))
display(test_df)

,seq_id,protein_sequence,pH,data_source,edit_type,edit_idx,wildtype_aa,mutant_aa,sasa
0,31390,VPVNPEPDATSVENVAEKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,substitution,16,L,E,170.882740
1,31391,VPVNPEPDATSVENVAKKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,substitution,16,L,K,170.882740
2,31392,VPVNPEPDATSVENVAKTGSGDSQSDPIKADLEVKGQSALPFDVDC...,8,Novozymes,deletion,16,L,-1,170.882740
3,31393,VPVNPEPDATSVENVALCTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,substitution,17,K,C,71.392633
4,31394,VPVNPEPDATSVENVALFTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,substitution,17,K,F,71.392633
...,...,...,...,...,...,...,...,...,...
2408,33798,VPVNPEPDATSVENVILKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,substitution,15,A,I,55.403077
2409,33799,VPVNPEPDATSVENVLLKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,substitution,15,A,L,55.403077
2410,33800,VPVNPEPDATSVENVNLKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,substitution,15,A,N,55.403077
2411,33801,VPVNPEPDATSVENVPLKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,substitution,15,A,P,55.403077


<br>

<b>Random Other Stuff For Fun</b>

In [4]:
from Bio.SeqUtils.ProtParam import ProteinAnalysis

test_df["stability_analysis"]   = test_df["protein_sequence"].apply(lambda x: ProteinAnalysis(x).instability_index())
test_df["aromaticity_analysis"] = test_df["protein_sequence"].apply(lambda x: ProteinAnalysis(x).aromaticity())
test_df["isoelectric_analysis"] = test_df["protein_sequence"].apply(lambda x: ProteinAnalysis(x).isoelectric_point())
test_df["charge_analysis"]      = test_df["protein_sequence"].apply(lambda x: ProteinAnalysis(x).charge_at_pH(8.0))
test_df["helix_analysis"]       = test_df["protein_sequence"].apply(lambda x: ProteinAnalysis(x).secondary_structure_fraction()[0])
test_df["turn_analysis"]        = test_df["protein_sequence"].apply(lambda x: ProteinAnalysis(x).secondary_structure_fraction()[1])
test_df["sheet_analysis"]       = test_df["protein_sequence"].apply(lambda x: ProteinAnalysis(x).secondary_structure_fraction()[2])
test_df["mec_analysis_1"]       = test_df["protein_sequence"].apply(lambda x: ProteinAnalysis(x).molar_extinction_coefficient()[0])
test_df["mec_analysis_2"]       = test_df["protein_sequence"].apply(lambda x: ProteinAnalysis(x).molar_extinction_coefficient()[1])
test_df["gravy_analysis"]       = test_df["protein_sequence"].apply(lambda x: ProteinAnalysis(x).gravy())
display(test_df[[_c for _c in test_df.columns if "analysis" in _c]])

,stability_analysis,aromaticity_analysis,isoelectric_analysis,charge_analysis,helix_analysis,turn_analysis,sheet_analysis,mec_analysis_1,mec_analysis_2,gravy_analysis
0,43.819955,0.099548,8.751535,2.558953,0.230769,0.330317,0.180995,41940,42190,-0.771041
1,43.819955,0.099548,9.024494,4.548770,0.230769,0.330317,0.176471,41940,42190,-0.772851
2,43.973682,0.100000,8.907484,3.558671,0.231818,0.331818,0.177273,41940,42190,-0.758636
3,45.295068,0.099548,8.699638,2.477663,0.235294,0.330317,0.180995,41940,42190,-0.709050
4,43.819955,0.104072,8.761012,2.568572,0.239819,0.330317,0.180995,41940,42190,-0.707692
...,...,...,...,...,...,...,...,...,...,...
2408,44.307285,0.099548,8.907484,3.558671,0.239819,0.330317,0.176471,41940,42190,-0.725792
2409,43.435792,0.099548,8.907484,3.558671,0.239819,0.330317,0.180995,41940,42190,-0.728959
2410,43.435792,0.099548,8.907484,3.558671,0.235294,0.334842,0.176471,41940,42190,-0.761991
2411,44.307285,0.099548,8.907484,3.558671,0.235294,0.334842,0.176471,41940,42190,-0.753394


<br>

<b>Get Depth (RD & CA)</b>

In [6]:
def get_depth_maps(pdb_path="/kaggle/input/novozymes-enzyme-stability-prediction/wildtype_structure_prediction_af2.pdb"):
    # Get residue depth model and use to calculate rd and ca depth
    model = pdb_parser.get_structure("", pdb_path)[0]
    rd = ResidueDepth(model)
    rd, ca = list(zip(*[(x[1][0], x[1][1]) for x in rd]))
    
    # Create mappings
    return {i:x for i,x in enumerate(rd)}, {i:x for i,x in enumerate(ca)}

    
rd_idx_2_depth, ca_idx_2_depth = get_depth_maps()    
test_df["rd"] = test_df["edit_idx"].apply(lambda x: rd_idx_2_depth.get(x, 0.0))
test_df["ca"] = test_df["edit_idx"].apply(lambda x: ca_idx_2_depth.get(x, 0.0))
test_df

,seq_id,protein_sequence,pH,data_source,edit_type,edit_idx,wildtype_aa,mutant_aa,sasa,stability_analysis,...,isoelectric_analysis,charge_analysis,helix_analysis,turn_analysis,sheet_analysis,mec_analysis_1,mec_analysis_2,gravy_analysis,rd,ca
0,31390,VPVNPEPDATSVENVAEKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,substitution,16,L,E,170.882740,43.819955,...,8.751535,2.558953,0.230769,0.330317,0.180995,41940,42190,-0.771041,1.511599,2.235219
1,31391,VPVNPEPDATSVENVAKKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,substitution,16,L,K,170.882740,43.819955,...,9.024494,4.548770,0.230769,0.330317,0.176471,41940,42190,-0.772851,1.511599,2.235219
2,31392,VPVNPEPDATSVENVAKTGSGDSQSDPIKADLEVKGQSALPFDVDC...,8,Novozymes,deletion,16,L,-1,170.882740,43.973682,...,8.907484,3.558671,0.231818,0.331818,0.177273,41940,42190,-0.758636,1.511599,2.235219
3,31393,VPVNPEPDATSVENVALCTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,substitution,17,K,C,71.392633,45.295068,...,8.699638,2.477663,0.235294,0.330317,0.180995,41940,42190,-0.709050,1.692059,2.382093
4,31394,VPVNPEPDATSVENVALFTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,substitution,17,K,F,71.392633,43.819955,...,8.761012,2.568572,0.239819,0.330317,0.180995,41940,42190,-0.707692,1.692059,2.382093
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2408,33798,VPVNPEPDATSVENVILKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,substitution,15,A,I,55.403077,44.307285,...,8.907484,3.558671,0.239819,0.330317,0.176471,41940,42190,-0.725792,1.520372,1.999637
2409,33799,VPVNPEPDATSVENVLLKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,substitution,15,A,L,55.403077,43.435792,...,8.907484,3.558671,0.239819,0.330317,0.180995,41940,42190,-0.728959,1.520372,1.999637
2410,33800,VPVNPEPDATSVENVNLKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,substitution,15,A,N,55.403077,43.435792,...,8.907484,3.558671,0.235294,0.334842,0.176471,41940,42190,-0.761991,1.520372,1.999637
2411,33801,VPVNPEPDATSVENVPLKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,substitution,15,A,P,55.403077,44.307285,...,8.907484,3.558671,0.235294,0.334842,0.176471,41940,42190,-0.753394,1.520372,1.999637


<br>

<b>Min-Max Normalization & Combine RD&CA</b>
* Will Operate Similar to Rank But Visually Easier to Interpret
* For combination (**`rd_ca_norm`**) we give:
    * <b>75%</b> weight to <b>Residue (RD)</b>
    * <b>25%</b> weight to <b>Chain (CA)</b>

In [7]:
def min_max_norm(x, invert=False):
    """ x is np array or series """
    scaled = ((x-x.min())/(x.max()-x.min()))
    if invert: scaled = 1-scaled
    return scaled
    
test_df["sasa_norm"] = min_max_norm(test_df["sasa"])
test_df["rd_norm"] = min_max_norm(test_df["rd"], invert=True)
test_df["ca_norm"] = min_max_norm(test_df["ca"], invert=True)
test_df["rd_ca_norm"] = (test_df["rd_norm"]*1.5 + test_df["ca_norm"]*0.5)/2
test_df

,seq_id,protein_sequence,pH,data_source,edit_type,edit_idx,wildtype_aa,mutant_aa,sasa,stability_analysis,...,sheet_analysis,mec_analysis_1,mec_analysis_2,gravy_analysis,rd,ca,sasa_norm,rd_norm,ca_norm,rd_ca_norm
0,31390,VPVNPEPDATSVENVAEKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,substitution,16,L,E,170.882740,43.819955,...,0.180995,41940,42190,-0.771041,1.511599,2.235219,0.801622,0.804002,0.723461,0.783866
1,31391,VPVNPEPDATSVENVAKKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,substitution,16,L,K,170.882740,43.819955,...,0.176471,41940,42190,-0.772851,1.511599,2.235219,0.801622,0.804002,0.723461,0.783866
2,31392,VPVNPEPDATSVENVAKTGSGDSQSDPIKADLEVKGQSALPFDVDC...,8,Novozymes,deletion,16,L,-1,170.882740,43.973682,...,0.177273,41940,42190,-0.758636,1.511599,2.235219,0.801622,0.804002,0.723461,0.783866
3,31393,VPVNPEPDATSVENVALCTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,substitution,17,K,C,71.392633,45.295068,...,0.180995,41940,42190,-0.709050,1.692059,2.382093,0.334907,0.780603,0.705289,0.761774
4,31394,VPVNPEPDATSVENVALFTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,substitution,17,K,F,71.392633,43.819955,...,0.180995,41940,42190,-0.707692,1.692059,2.382093,0.334907,0.780603,0.705289,0.761774
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2408,33798,VPVNPEPDATSVENVILKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,substitution,15,A,I,55.403077,44.307285,...,0.176471,41940,42190,-0.725792,1.520372,1.999637,0.259899,0.802864,0.752607,0.790300
2409,33799,VPVNPEPDATSVENVLLKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,substitution,15,A,L,55.403077,43.435792,...,0.180995,41940,42190,-0.728959,1.520372,1.999637,0.259899,0.802864,0.752607,0.790300
2410,33800,VPVNPEPDATSVENVNLKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,substitution,15,A,N,55.403077,43.435792,...,0.176471,41940,42190,-0.761991,1.520372,1.999637,0.259899,0.802864,0.752607,0.790300
2411,33801,VPVNPEPDATSVENVPLKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,substitution,15,A,P,55.403077,44.307285,...,0.176471,41940,42190,-0.753394,1.520372,1.999637,0.259899,0.802864,0.752607,0.790300


<br>

<b>DeMaSk for Mutation Weighting</b>

In [8]:
# 1.1 – Create substitution matrix for each position from DeMaSk data (apologies for the dump file names)
demask_sub_matrix = pd.read_csv("../input/demask-directional-substitution-matrix/inpsseq_mutation_ddg - Sheet2.csv")
demask_tuple_sub_map = {}
for _, _row in demask_sub_matrix.iterrows():
    demask_tuple_sub_map[(_row["pos"]-1, _row["WT"], _row["var"])] = _row["score"]
    demask_tuple_sub_map[(_row["pos"]-1, _row["var"], _row["WT"])] = _row["score"]

# 1.2 – Conduct matrix substitution
#   --> First we create a tuple that has the wildtype amino acid and the 
#       mutant amino acid to access the substitution matrix
#   --> Second we access the substitution matrix and replace with the respective score
#       and in cases where no respective score is found we mark it to be updated later
test_df["demask_sub_matrix_tuple"] = test_df[["edit_idx", "wildtype_aa", "mutant_aa"]].apply(tuple, axis=1)
test_df["demask"] = test_df["demask_sub_matrix_tuple"].apply(lambda _mutant_tuple: demask_tuple_sub_map.get(_mutant_tuple, pd.NA))
test_df = test_df.drop(columns=["demask_sub_matrix_tuple"])

# 1.3 – Fill in the missing data with default values for now
#   --> We aim to make this as similar to the -10.0 that was previously used as possible
#       the most similar value considering the different scale would be --> -0.75
#   --> No change remains as 0.0
DEMASK_DEFAULT__DELETION__MATRIXSCORE = -0.75 # This is about 1.25 times larger than the minimum value (just like -10 in the original)
DEMASK_DEFAULT__NO_CHANGE__MATRIXSCORE = 0.0
test_df.loc[test_df['edit_type']=="deletion", 'demask'] = DEMASK_DEFAULT__DELETION__MATRIXSCORE
test_df.loc[test_df['edit_type']==-1, 'demask'] = DEMASK_DEFAULT__NO_CHANGE__MATRIXSCORE
test_df["demask"] = test_df["demask"].astype(float)
test_df["demask_norm"] = min_max_norm(test_df["demask"])

<br>

<b>Create Combined Score Column (**`tm`**) And Show Relevant Correlation Values</b>
* Note that we technically are weighting DeMaSk as twice as important as the RD/SASA (RD or RD_CA) values as it is simply a multiplier
    * i.e. **`RD + SASA  =  SCORE`**
    * i.e. **` DEMASK*(RD+SASA)  =  DEMASK*RD + DEMASK*SASA  =  WTD_SCORE`**    
        * 2 * DEMASK
        * 1 * RD
        * 1 * SASA

In [9]:
use_cols, weight_cols = ["sasa_norm", "rd_ca_norm"], ["demask_norm"]
test_df["tm"] = (test_df[use_cols].mean(axis=1)) * (test_df[weight_cols].mean(axis=1))
display(test_df[use_cols+weight_cols+["tm",]].corr("spearman"))

,sasa_norm,rd_ca_norm,demask_norm,tm
sasa_norm,1.000000,0.789203,0.497201,0.840307
rd_ca_norm,0.789203,1.000000,0.432422,0.748124
demask_norm,0.497201,0.432422,1.000000,0.818479
tm,0.840307,0.748124,0.818479,1.000000


<br>

<b>Make submission file</b>

In [10]:
ss_df["tm"] = test_df["tm"]
ss_df.to_csv("rd_cd_sasa_demaskweighting.csv", index=False)